# Pyterrier Notebook for evaluating project test runs

### Install dependencies 

In [2]:
!pip3 install python-terrier tira==0.0.88 ir_datasets trectools

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trectools: filename=trectools-0.0.49-py3-none-any.whl size=27122 sha256=4db7d9afac6f1647b726ee9f951ca840ad77f1a031e37d819bd25cee060599ba
  Stored in directory: /home/codespace/.cache/pip/wheels/e1/95/fc/d4a374ba4444ef75666828722b1cd44df22226885c0f111c92
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=a2a778addaf45fc403fcbe58cf3d1be65a649b8757dc7bbce8a3d119dc9c0308
  Stored in directory: /home/codespace/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built trectools bs4


Load imports and datasets

In [4]:
from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()


def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-jena-leipzig-wise-2023', dataset, truth_dataset=True) + '/qrels.txt')

training_qrels = load_qrels('training-20231104-training')
validation_qrels = load_qrels('validation-20231104-training')

No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.
No settings given in /home/codespace/.tira/.tira-settings.json. I will use defaults.


Define function to read runs from from TREC run file

In [9]:
def evaluate_run(run_url, qrels):
    run = TrecRun(run_url)
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'run_url': run_url,
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank()
    }

Convert to pandas dataframe and sort by ndcg at 10

In [10]:
df = []
for r in glob('runs_milestone3/*.txt'):
    df += [evaluate_run(r, training_qrels)]
df = pd.DataFrame(df)
df.sort_values('nDCG@10', ascending=False)

/home/codespace/.python/current/lib/python3.10/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/home/codespace/.python/current/lib/python3.10/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/home/codespace/.python/current/lib/python3.10/site-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a f

,run,run_url,nDCG@10,nDCG@10 (unjudgedRemoved),MAP,MRR
3,bm25-baseline,runs_milestone3/run_snowball_validation_optimi...,0.181197,0.537554,0.121498,0.265651
1,bm25-baseline,runs_milestone3/run_with_snowball_stemmer.txt,0.178046,0.536264,0.119095,0.264599
0,bm25-baseline,runs_milestone3/run_with_porter_stemmer.txt,0.177380,0.536716,0.118656,0.262814
2,bm25-baseline,runs_milestone3/run_without_stemmer.txt,0.174777,0.529697,0.116601,0.264982
